In [84]:
# 필요 패키지 설치 (처음 1회만 실행)
%pip install -q transformers torch
%pip install hdbscan
%pip install -q python-dotenv
%pip install -q pybloom-live

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from dotenv import load_dotenv
import torch
from transformers import pipeline

# .env 파일에서 설정 로드
load_dotenv()

# 설정값 가져오기
BERT_MODEL_NAME = os.getenv('BERT_MODEL_NAME', 'bert-base-uncased')
CLUSTERING_ALPHA = float(os.getenv('CLUSTERING_ALPHA', '1.0'))
CLUSTERING_METHOD = os.getenv('CLUSTERING_METHOD', 'dbscan')
CLUSTERING_RANDOM_STATE = int(os.getenv('CLUSTERING_RANDOM_STATE', '42'))
KEEP_PREFIXES_STR = os.getenv('KEEP_PREFIXES', 'processName,eventName,syscall,args,executable,returnValue')
BATCH_SIZE = int(os.getenv('BATCH_SIZE', '32'))
SHUFFLE = os.getenv('SHUFFLE', 'False').lower() == 'true'
TOP_N_CLUSTERS = int(os.getenv('TOP_N_CLUSTERS', '10'))
N_GRAM_SIZE = int(os.getenv('N_GRAM_SIZE_CONF', '2'))
BLOOM_FILTER_ERROR_RATE = float(os.getenv('BLOOM_FILTER_ERROR_RATE', '0.001'))

# keep_prefixes를 튜플로 변환
keep_prefixes = tuple([prefix.strip() for prefix in KEEP_PREFIXES_STR.split(',')])

print("=" * 80)
print("설정 로드 완료")
print("=" * 80)
print(f"BERT 모델: {BERT_MODEL_NAME}")
print(f"클러스터링 Alpha: {CLUSTERING_ALPHA}")
print(f"클러스터링 Method: {CLUSTERING_METHOD}")
print(f"Random State: {CLUSTERING_RANDOM_STATE}")
print(f"Keep Prefixes: {keep_prefixes}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Shuffle: {SHUFFLE}")
print(f"Top N Clusters: {TOP_N_CLUSTERS}")
print(f"N-gram Size: {N_GRAM_SIZE}")
print(f"Bloom Filter Error Rate: {BLOOM_FILTER_ERROR_RATE} ({BLOOM_FILTER_ERROR_RATE*100:.3f}%)")
print("=" * 80)

# BERT 다운로드 및 로드
extractor = pipeline(
    "feature-extraction",
    model=BERT_MODEL_NAME,
    tokenizer=BERT_MODEL_NAME,
    return_tensors=True,
)

def mean_pool(token_embeddings):
    # token_embeddings: (batch, seq_len, hidden_size)
    return token_embeddings.mean(dim=1)

python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 3
python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 5
python-dotenv could not parse statement starting at line 6
python-dotenv could not parse statement starting at line 7


설정 로드 완료
BERT 모델: bert-base-uncased
클러스터링 Alpha: 0.5
클러스터링 Method: dbscan
Random State: 42
Keep Prefixes: ('processName', 'eventName', 'syscall', 'args', 'executable', 'returnValue')
Batch Size: 32
Shuffle: False
Top N Clusters: 10
N-gram Size: 5


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 487.14it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [75]:
import json
import os
from torch.utils.data import Dataset, DataLoader
from typing import Literal

class TraceeDataset(Dataset):
    def __init__(self, data_path, filter_func=None):
        """
        Args:
            data_path: JSON Lines 파일 경로
            filter_func: 텍스트 필터링 함수 (선택사항)
        """
        self.data = []
        self.data_path = data_path  # 원본 데이터 경로 저장
        self.filter_func = filter_func
        self.original_data = []  # 원본 JSON 데이터 저장
        
        with open(data_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():
                    try:
                        json_obj = json.loads(line)
                        # 원본 데이터 저장
                        self.original_data.append(json_obj)
                        # JSON 객체를 문자열로 변환
                        text = json.dumps(json_obj, ensure_ascii=False)
                        if filter_func:
                            text = filter_func(text)
                        self.data.append(text)
                    except json.JSONDecodeError:
                        continue
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def getDataLoader(
    split: Literal["train", "validation"],
    data_type: Literal["attack", "normal"],
    batch_size: int = 32,
    shuffle: bool = True,
    filter_func=None,
    num_workers: int = 0
):
    """
    데이터 로더를 생성하는 함수
    
    Args:
        split: "train" 또는 "validation"
        data_type: "attack" 또는 "normal"
        batch_size: 배치 크기
        shuffle: 셔플 여부 (기본값: True)
        filter_func: 텍스트 필터링 함수 (선택사항)
        num_workers: 데이터 로딩 워커 수 (기본값: 0)
    
    Returns:
        DataLoader 객체
    """
    # 파일명 구성
    prefix = "tr" if split == "train" else "val"
    filename = f"{prefix}_{data_type}_tracee.json"
    data_path = os.path.join("data", filename)
    
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"데이터 파일을 찾을 수 없습니다: {data_path}")
    
    # Dataset 생성
    dataset = TraceeDataset(data_path, filter_func=filter_func)
    
    # DataLoader 생성
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        collate_fn=lambda x: x  # 배치를 리스트로 반환
    )
    
    return dataloader

In [76]:
import re

# 유사도를 높이는 공통/식별값(IDs, timestamp 등)을 제거하고 핵심 필드만 남깁니다.
# keep_prefixes는 .env 파일에서 로드됨 (셀 1에서 설정됨)

kv_pattern = re.compile(r'"([^"]+)":\s*"([^"]*)"')

def filter_text(raw_text, keep_keys=None):
    """
    텍스트에서 지정된 키만 남기는 필터링 함수
    
    Args:
        raw_text: 원본 JSON 문자열
        keep_keys: 유지할 키의 튜플 (기본값: 전역 keep_prefixes)
    
    Returns:
        필터링된 텍스트 문자열
    """
    if keep_keys is None:
        keep_keys = keep_prefixes
    pairs = kv_pattern.findall(raw_text)
    kept = []
    for key, value in pairs:
        if key.startswith(keep_keys):
            kept.append(f"{key}={value}")
    return " ".join(kept)


In [77]:
from tqdm import tqdm

# Train normal 데이터 로더 생성 (.env에서 설정값 사용)
train_normal_loader = getDataLoader(
    "train", 
    "normal", 
    batch_size=BATCH_SIZE, 
    shuffle=SHUFFLE, 
    filter_func=filter_text
)

# 모든 데이터를 임베딩으로 변환하여 리스트에 저장
embeddings_list = []

print(f"총 데이터 개수: {len(train_normal_loader.dataset)}")
print("임베딩 생성 중...")

for batch_idx, batch_texts in enumerate(tqdm(train_normal_loader, desc="임베딩 생성")):
    # 배치 내 각 텍스트에 대해 임베딩 생성
    for text in batch_texts:
        # 문장 그대로 입력하면 내부에서 토크나이즈가 처리됨
        outputs = extractor(text, padding=True, truncation=True)  # Shape: (1, seq_len, hidden_size)
        # 평균 풀링하여 문장 임베딩 생성
        pooled_embedding = mean_pool(outputs)  # Shape: (1, hidden_size)
        # 배치 차원 제거하고 리스트에 추가
        embeddings_list.append(pooled_embedding.squeeze(0))  # Shape: (hidden_size,)

# 리스트를 텐서로 변환 (선택사항)
embeddings_tensor = torch.stack(embeddings_list)  # Shape: (num_samples, hidden_size)

print(f"\n완료! 임베딩 리스트 길이: {len(embeddings_list)}")
print(f"임베딩 텐서 shape: {embeddings_tensor.shape}")

총 데이터 개수: 721
임베딩 생성 중...


임베딩 생성: 100%|██████████| 23/23 [01:31<00:00,  3.99s/it]


완료! 임베딩 리스트 길이: 721
임베딩 텐서 shape: torch.Size([721, 768])


In [78]:
import numpy as np
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, MeanShift
from sklearn.metrics import silhouette_score

# HDBSCAN은 별도 설치 필요: pip install hdbscan
try:
    import hdbscan
    HDBSCAN_AVAILABLE = True
except ImportError:
    HDBSCAN_AVAILABLE = False
    print("HDBSCAN을 사용하려면 'pip install hdbscan'을 실행하세요.")

def cluster_embeddings(embeddings, alpha=1.0, method='kmeans', random_state=42):
    """
    임베딩을 클러스터링하는 함수
    
    Args:
        embeddings: 임베딩 텐서 또는 numpy 배열 (n_samples, embedding_dim)
        alpha: 클러스터링 비율 조절 파라미터 (0.1 ~ 2.0)
              - alpha가 낮을수록 적은 클러스터 (큰 그룹)
              - alpha가 높을수록 많은 클러스터 (세밀한 그룹)
        method: 클러스터링 방법 ('kmeans' 또는 'dbscan')
        random_state: 랜덤 시드
    
    Returns:
        labels: 각 샘플의 클러스터 레이블 (numpy array)
        n_clusters: 생성된 클러스터 수
        cluster_info: 클러스터 정보 딕셔너리
    """
    # 텐서를 numpy 배열로 변환
    if isinstance(embeddings, torch.Tensor):
        embeddings_np = embeddings.detach().cpu().numpy()
    else:
        embeddings_np = np.array(embeddings)
    
    n_samples = embeddings_np.shape[0]
    
    if method == 'kmeans':
        # alpha 값에 따라 클러스터 수 결정
        # alpha=0.1 -> sqrt(n_samples/10), alpha=1.0 -> sqrt(n_samples), alpha=2.0 -> sqrt(n_samples*10)
        base_clusters = int(np.sqrt(n_samples))
        n_clusters = max(2, int(base_clusters * alpha))
        
        # K-means 클러스터링
        kmeans = KMeans(n_clusters=n_clusters, random_state=random_state, n_init=10)
        labels = kmeans.fit_predict(embeddings_np)
        
        # 실루엣 점수 계산
        if n_clusters > 1:
            silhouette_avg = silhouette_score(embeddings_np, labels)
        else:
            silhouette_avg = 0.0
        
        cluster_info = {
            'method': 'kmeans',
            'n_clusters': n_clusters,
            'silhouette_score': silhouette_avg,
            'inertia': kmeans.inertia_,
            'centers': kmeans.cluster_centers_
        }
        
    elif method == 'dbscan':
        # alpha 값에 따라 eps와 min_samples 조절
        # alpha가 낮을수록 큰 클러스터 (큰 eps), 높을수록 작은 클러스터 (작은 eps)
        # 기본 eps는 데이터의 평균 거리의 일정 비율
        from sklearn.neighbors import NearestNeighbors
        neighbors = NearestNeighbors(n_neighbors=min(10, n_samples))
        neighbors_fit = neighbors.fit(embeddings_np)
        distances, indices = neighbors_fit.kneighbors(embeddings_np)
        distances = np.sort(distances, axis=0)
        distances = distances[:, 1]
        
        # 기본 eps는 거리의 중앙값
        base_eps = np.median(distances)
        # alpha에 따라 조절: alpha=0.1 -> 큰 eps, alpha=2.0 -> 작은 eps
        eps = base_eps / alpha
        
        min_samples = max(3, int(np.log(n_samples) * alpha))
        
        # DBSCAN 클러스터링
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(embeddings_np)
        
        # 클러스터 수 계산 (노이즈 제외)
        unique_labels = set(labels)
        n_clusters = len(unique_labels) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        
        # 실루엣 점수 계산 (노이즈 제외)
        if n_clusters > 1:
            mask = labels != -1
            if mask.sum() > 1:
                silhouette_avg = silhouette_score(embeddings_np[mask], labels[mask])
            else:
                silhouette_avg = 0.0
        else:
            silhouette_avg = 0.0
        
        cluster_info = {
            'method': 'dbscan',
            'n_clusters': n_clusters,
            'n_noise': n_noise,
            'silhouette_score': silhouette_avg,
            'eps': eps,
            'min_samples': min_samples
        }
    
    elif method == 'hdbscan':
        # HDBSCAN: DBSCAN의 개선 버전, 클러스터 개수 자동 결정
        # 밀도 기반 계층적 클러스터링, 다양한 크기의 클러스터 처리에 우수
        if not HDBSCAN_AVAILABLE:
            raise ImportError("HDBSCAN이 설치되지 않았습니다. 'pip install hdbscan'을 실행하세요.")
        
        # alpha 값에 따라 min_cluster_size와 min_samples 조절
        # alpha가 낮을수록 큰 클러스터, 높을수록 작은 클러스터
        min_cluster_size = max(2, int(n_samples / (10 * alpha)))
        min_samples = max(3, int(np.log(n_samples) * alpha))
        
        clusterer = hdbscan.HDBSCAN(
            min_cluster_size=min_cluster_size,
            min_samples=min_samples,
            cluster_selection_epsilon=0.0
        )
        labels = clusterer.fit_predict(embeddings_np)
        
        # 클러스터 수 계산 (노이즈 제외)
        unique_labels = set(labels)
        n_clusters = len(unique_labels) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        
        # 실루엣 점수 계산 (노이즈 제외)
        if n_clusters > 1:
            mask = labels != -1
            if mask.sum() > 1:
                silhouette_avg = silhouette_score(embeddings_np[mask], labels[mask])
            else:
                silhouette_avg = 0.0
        else:
            silhouette_avg = 0.0
        
        cluster_info = {
            'method': 'hdbscan',
            'n_clusters': n_clusters,
            'n_noise': n_noise,
            'silhouette_score': silhouette_avg,
            'min_cluster_size': min_cluster_size,
            'min_samples': min_samples,
            'cluster_probabilities': clusterer.probabilities_
        }
    
    elif method == 'agglomerative':
        # Agglomerative Clustering: 거리 threshold로 클러스터 개수 자동 결정
        # 계층적 클러스터링, 거리 기반으로 클러스터 수 자동 결정
        
        # 거리 행렬 계산
        from sklearn.metrics import pairwise_distances
        distances = pairwise_distances(embeddings_np)
        
        # alpha 값에 따라 거리 threshold 조절
        # alpha가 낮을수록 큰 클러스터 (큰 threshold), 높을수록 작은 클러스터 (작은 threshold)
        base_threshold = np.percentile(distances[distances > 0], 50)  # 중앙값
        distance_threshold = base_threshold / alpha
        
        # Agglomerative Clustering
        clustering = AgglomerativeClustering(
            n_clusters=None,
            distance_threshold=distance_threshold,
            linkage='ward'
        )
        labels = clustering.fit_predict(embeddings_np)
        
        # 클러스터 수 계산
        unique_labels = set(labels)
        n_clusters = len(unique_labels)
        
        # 실루엣 점수 계산
        if n_clusters > 1:
            silhouette_avg = silhouette_score(embeddings_np, labels)
        else:
            silhouette_avg = 0.0
        
        cluster_info = {
            'method': 'agglomerative',
            'n_clusters': n_clusters,
            'silhouette_score': silhouette_avg,
            'distance_threshold': distance_threshold
        }
    
    elif method == 'meanshift':
        # Mean Shift: 클러스터 개수 자동 결정, 밀도 기반
        # bandwidth(대역폭)에 따라 클러스터 수 결정
        
        # alpha 값에 따라 bandwidth 조절
        # alpha가 낮을수록 큰 클러스터 (큰 bandwidth), 높을수록 작은 클러스터 (작은 bandwidth)
        from sklearn.cluster import estimate_bandwidth
        from sklearn.metrics import pairwise_distances
        
        # 거리 기반으로 bandwidth 추정 (더 안정적)
        try:
            # 샘플링하여 거리 계산 (대용량 데이터 처리)
            sample_size = min(500, n_samples)
            if n_samples > sample_size:
                sample_indices = np.random.choice(n_samples, sample_size, replace=False)
                sample_data = embeddings_np[sample_indices]
            else:
                sample_data = embeddings_np
            
            # 거리 행렬 계산
            distances = pairwise_distances(sample_data)
            # 0이 아닌 거리의 중앙값 사용
            non_zero_distances = distances[distances > 0]
            if len(non_zero_distances) > 0:
                base_bandwidth = np.median(non_zero_distances)
            else:
                # fallback: estimate_bandwidth 사용
                base_bandwidth = estimate_bandwidth(embeddings_np, quantile=0.3, n_samples=sample_size)
        except:
            # fallback: estimate_bandwidth 사용
            base_bandwidth = estimate_bandwidth(embeddings_np, quantile=0.3, n_samples=min(500, n_samples))
        
        # alpha에 따라 조절: alpha가 낮을수록 큰 bandwidth
        # 최소 bandwidth 보장 (너무 작으면 에러 발생)
        bandwidth = max(base_bandwidth / alpha, base_bandwidth * 0.1)
        
        # Mean Shift 클러스터링 (에러 처리 포함)
        try:
            ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, max_iter=300)
            labels = ms.fit_predict(embeddings_np)
            
            # 클러스터 수 계산
            unique_labels = set(labels)
            n_clusters = len(unique_labels)
            
            # 실루엣 점수 계산
            if n_clusters > 1:
                silhouette_avg = silhouette_score(embeddings_np, labels)
            else:
                silhouette_avg = 0.0
            
            cluster_info = {
                'method': 'meanshift',
                'n_clusters': n_clusters,
                'silhouette_score': silhouette_avg,
                'bandwidth': bandwidth
            }
        except ValueError as e:
            # bandwidth가 너무 작아서 에러 발생 시, 자동으로 증가시켜 재시도
            if "bandwidth" in str(e).lower():
                # bandwidth를 2배로 증가
                bandwidth = bandwidth * 2
                try:
                    ms = MeanShift(bandwidth=bandwidth, bin_seeding=False, max_iter=300)
                    labels = ms.fit_predict(embeddings_np)
                    unique_labels = set(labels)
                    n_clusters = len(unique_labels)
                    if n_clusters > 1:
                        silhouette_avg = silhouette_score(embeddings_np, labels)
                    else:
                        silhouette_avg = 0.0
                    cluster_info = {
                        'method': 'meanshift',
                        'n_clusters': n_clusters,
                        'silhouette_score': silhouette_avg,
                        'bandwidth': bandwidth,
                        'note': 'bandwidth 자동 조정됨'
                    }
                except:
                    # 여전히 실패하면 모든 포인트를 하나의 클러스터로
                    labels = np.zeros(n_samples, dtype=int)
                    cluster_info = {
                        'method': 'meanshift',
                        'n_clusters': 1,
                        'silhouette_score': 0.0,
                        'bandwidth': bandwidth,
                        'note': 'bandwidth 조정 실패, 모든 포인트를 하나의 클러스터로 처리'
                    }
            else:
                raise
    
    else:
        raise ValueError(f"지원하지 않는 클러스터링 방법: {method}. "
                        f"사용 가능한 방법: 'kmeans', 'dbscan', 'hdbscan', 'agglomerative', 'meanshift'")
    
    return labels, n_clusters, cluster_info

# 사용 예시 및 알고리즘 비교
if 'embeddings_tensor' in globals():
    print("=" * 80)
    print("클러스터 개수를 미리 지정하지 않는 알고리즘들 비교")
    print("=" * 80)
    
    # DBSCAN 테스트
    print("\n[DBSCAN] - 밀도 기반, 노이즈 포인트 식별 가능")
    print("-" * 80)
    for alpha in [0.5, 1.0, 1.5]:
        labels, n_clusters, info = cluster_embeddings(
            embeddings_tensor, 
            alpha=alpha, 
            method='dbscan'
        )
        print(f"Alpha: {alpha:.1f} -> 클러스터 수: {n_clusters}, 노이즈: {info['n_noise']}, 실루엣: {info['silhouette_score']:.4f}")
    
    # HDBSCAN 테스트 (가장 추천!)
    if HDBSCAN_AVAILABLE:
        print("\n[HDBSCAN] - DBSCAN 개선 버전, 다양한 크기 클러스터 처리 우수 (추천!)")
        print("-" * 80)
        for alpha in [0.5, 1.0, 1.5]:
            try:
                labels, n_clusters, info = cluster_embeddings(
                    embeddings_tensor, 
                    alpha=alpha, 
                    method='hdbscan'
                )
                print(f"Alpha: {alpha:.1f} -> 클러스터 수: {n_clusters}, 노이즈: {info['n_noise']}, 실루엣: {info['silhouette_score']:.4f}")
            except Exception as e:
                print(f"Alpha: {alpha:.1f} -> 오류: {e}")
    else:
        print("\n[HDBSCAN] - 설치 필요: pip install hdbscan")
    
    # Agglomerative Clustering 테스트
    print("\n[Agglomerative Clustering] - 계층적, 거리 threshold 기반")
    print("-" * 80)
    for alpha in [0.5, 1.0, 1.5]:
        labels, n_clusters, info = cluster_embeddings(
            embeddings_tensor, 
            alpha=alpha, 
            method='agglomerative'
        )
        print(f"Alpha: {alpha:.1f} -> 클러스터 수: {n_clusters}, 실루엣: {info['silhouette_score']:.4f}")
    
    # Mean Shift 테스트
    print("\n[Mean Shift] - 밀도 기반, bandwidth 조절")
    print("-" * 80)
    for alpha in [0.5, 1.0, 1.5]:
        labels, n_clusters, info = cluster_embeddings(
            embeddings_tensor, 
            alpha=alpha, 
            method='meanshift'
        )
        print(f"Alpha: {alpha:.1f} -> 클러스터 수: {n_clusters}, 실루엣: {info['silhouette_score']:.4f}")
    
    print("\n" + "=" * 80)
    print("추천: 클러스터 개수가 많고 미리 지정할 수 없다면 HDBSCAN 또는 DBSCAN 사용")
    print("=" * 80)
else:
    print("embeddings_tensor가 아직 생성되지 않았습니다. 먼저 셀 4를 실행해주세요.")

클러스터 개수를 미리 지정하지 않는 알고리즘들 비교

[DBSCAN] - 밀도 기반, 노이즈 포인트 식별 가능
--------------------------------------------------------------------------------
Alpha: 0.5 -> 클러스터 수: 10, 노이즈: 13, 실루엣: 1.0000
Alpha: 1.0 -> 클러스터 수: 7, 노이즈: 23, 실루엣: 1.0000
Alpha: 1.5 -> 클러스터 수: 2, 노이즈: 448, 실루엣: 1.0000

[HDBSCAN] - DBSCAN 개선 버전, 다양한 크기 클러스터 처리 우수 (추천!)
--------------------------------------------------------------------------------
Alpha: 0.5 -> 클러스터 수: 3, 노이즈: 56, 실루엣: 0.9076
Alpha: 1.0 -> 클러스터 수: 3, 노이즈: 56, 실루엣: 0.9076
Alpha: 1.5 -> 클러스터 수: 4, 노이즈: 3, 실루엣: 0.9120

[Agglomerative Clustering] - 계층적, 거리 threshold 기반
--------------------------------------------------------------------------------
Alpha: 0.5 -> 클러스터 수: 8, 실루엣: 0.9584
Alpha: 1.0 -> 클러스터 수: 12, 실루엣: 0.9782
Alpha: 1.5 -> 클러스터 수: 14, 실루엣: 0.9840

[Mean Shift] - 밀도 기반, bandwidth 조절
--------------------------------------------------------------------------------
Alpha: 0.5 -> 클러스터 수: 1, 실루엣: 0.0000
Alpha: 1.0 -> 클러스터 수: 1, 실루엣: 0.0000
Alpha: 1.5 -

In [79]:
import json
from collections import defaultdict
import pandas as pd

class ClusterTracer:
    """
    클러스터링 결과와 원본 데이터를 매칭하여 추적하는 클래스
    """
    def __init__(self, dataloader, cluster_labels, filter_func=None):
        """
        Args:
            dataloader: 데이터 로더 (원본 데이터 접근용)
            cluster_labels: 클러스터링 결과 레이블 (numpy array)
            filter_func: 필터링 함수 (원본 데이터 로드 시 사용)
        """
        self.dataloader = dataloader
        self.cluster_labels = cluster_labels
        self.filter_func = filter_func
        self.original_data = []
        self.cluster_to_indices = defaultdict(list)
        
        # 원본 데이터 로드
        self._load_original_data()
        
        # 클러스터별 인덱스 매핑
        self._map_clusters()
    
    def _load_original_data(self):
        """원본 JSON 데이터를 로드"""
        dataset = self.dataloader.dataset
        
        # TraceeDataset에서 원본 데이터 가져오기
        if hasattr(dataset, 'original_data'):
            self.original_data = dataset.original_data
        elif hasattr(dataset, 'data_path'):
            # 원본 파일에서 다시 로드
            data_path = dataset.data_path
            with open(data_path, 'r', encoding='utf-8') as f:
                for line in f:
                    if line.strip():
                        try:
                            json_obj = json.loads(line)
                            self.original_data.append(json_obj)
                        except json.JSONDecodeError:
                            continue
        else:
            # 데이터 로더에서 직접 가져오기 (필터링된 버전)
            for i in range(len(dataset)):
                text = dataset[i]
                # 필터링된 텍스트를 저장
                self.original_data.append({'filtered_text': text})
    
    def _map_clusters(self):
        """클러스터 레이블과 데이터 인덱스를 매핑"""
        for idx, label in enumerate(self.cluster_labels):
            self.cluster_to_indices[label].append(idx)
    
    def get_cluster_data(self, cluster_id):
        """
        특정 클러스터에 속한 모든 데이터 반환
        
        Args:
            cluster_id: 클러스터 ID
        
        Returns:
            해당 클러스터에 속한 데이터 리스트
        """
        if cluster_id not in self.cluster_to_indices:
            return []
        
        indices = self.cluster_to_indices[cluster_id]
        return [self.original_data[idx] for idx in indices]
    
    def get_cluster_summary(self):
        """
        클러스터별 요약 정보 반환
        
        Returns:
            클러스터 요약 정보 딕셔너리
        """
        summary = {}
        for cluster_id, indices in self.cluster_to_indices.items():
            summary[cluster_id] = {
                'count': len(indices),
                'indices': indices,
                'sample_data': self.original_data[indices[0]] if indices else None
            }
        return summary
    
    def print_cluster_summary(self, top_n=10):
        """
        클러스터 요약 정보 출력
        
        Args:
            top_n: 출력할 상위 클러스터 개수
        """
        # 클러스터를 크기 순으로 정렬
        sorted_clusters = sorted(
            self.cluster_to_indices.items(),
            key=lambda x: len(x[1]),
            reverse=True
        )
        
        print("=" * 80)
        print("클러스터 요약 정보")
        print("=" * 80)
        print(f"총 클러스터 수: {len(self.cluster_to_indices)}")
        print(f"총 데이터 수: {len(self.cluster_labels)}")
        print()
        
        for i, (cluster_id, indices) in enumerate(sorted_clusters[:top_n]):
            print(f"[클러스터 {cluster_id}] 데이터 개수: {len(indices)}")
            if indices:
                sample = self.original_data[indices[0]]
                # 주요 필드만 출력
                if isinstance(sample, dict):
                    key_fields = ['processName', 'eventName', 'syscall', 'eventId']
                    sample_info = {k: sample.get(k, 'N/A') for k in key_fields if k in sample}
                    print(f"  샘플 데이터: {sample_info}")
                else:
                    print(f"  샘플 데이터: {str(sample)[:200]}...")
            print()
    
    def get_cluster_statistics(self):
        """
        클러스터 통계 정보 반환 (DataFrame)
        
        Returns:
            클러스터 통계 DataFrame
        """
        stats = []
        for cluster_id, indices in self.cluster_to_indices.items():
            cluster_data = [self.original_data[idx] for idx in indices]
            
            # 주요 필드별 통계
            process_names = [d.get('processName', 'N/A') for d in cluster_data if isinstance(d, dict)]
            event_names = [d.get('eventName', 'N/A') for d in cluster_data if isinstance(d, dict)]
            syscalls = [d.get('syscall', 'N/A') for d in cluster_data if isinstance(d, dict)]
            
            stats.append({
                'cluster_id': cluster_id,
                'count': len(indices),
                'unique_processes': len(set(process_names)),
                'unique_events': len(set(event_names)),
                'unique_syscalls': len(set(syscalls)),
                'most_common_process': max(set(process_names), key=process_names.count) if process_names else 'N/A',
                'most_common_event': max(set(event_names), key=event_names.count) if event_names else 'N/A',
                'most_common_syscall': max(set(syscalls), key=syscalls.count) if syscalls else 'N/A',
            })
        
        return pd.DataFrame(stats).sort_values('count', ascending=False)
    
    def export_cluster_mapping(self, output_path='cluster_mapping.json'):
        """
        클러스터 매핑 정보를 JSON 파일로 저장
        각 인덱스에 번호와 실제 텍스트 데이터를 포함
        
        Args:
            output_path: 출력 파일 경로
        """
        mapping = {
            'total_clusters': len(self.cluster_to_indices),
            'total_samples': len(self.cluster_labels),
            'clusters': {}
        }
        
        for cluster_id, indices in self.cluster_to_indices.items():
            # 각 인덱스에 대해 번호와 실제 데이터를 매핑
            indexed_data = []
            for idx in indices:
                data = self.original_data[idx]
                indexed_data.append({
                    'index': idx,
                    'data': data
                })
            
            mapping['clusters'][str(cluster_id)] = {
                'count': len(indices),
                'indices': indices,  # 기존 인덱스 리스트도 유지
                'data': indexed_data,  # 번호와 실제 데이터 매핑
                'sample_data': self.original_data[indices[0]] if indices else None
            }
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(mapping, f, ensure_ascii=False, indent=2)
        
        print(f"클러스터 매핑 정보가 {output_path}에 저장되었습니다.")

In [80]:
# 클러스터링 결과와 원본 데이터 매칭 예시
if 'embeddings_tensor' in globals() and 'train_normal_loader' in globals():
    # 클러스터링 수행 (.env에서 설정값 사용)
    print("클러스터링 수행 중...")
    labels, n_clusters, info = cluster_embeddings(
        embeddings_tensor,
        alpha=CLUSTERING_ALPHA,
        method=CLUSTERING_METHOD,
        random_state=CLUSTERING_RANDOM_STATE
    )
    
    print(f"\n클러스터링 완료!")
    print(f"클러스터 수: {n_clusters}")
    print(f"노이즈 포인트: {info.get('n_noise', 0)}")
    print(f"실루엣 점수: {info['silhouette_score']:.4f}")
    
    # ClusterTracer 생성
    print("\n" + "=" * 80)
    print("클러스터 추적 객체 생성 중...")
    tracer = ClusterTracer(
        dataloader=train_normal_loader,
        cluster_labels=labels,
        filter_func=filter_text
    )
    
    # 클러스터 요약 정보 출력 (.env에서 설정값 사용)
    tracer.print_cluster_summary(top_n=TOP_N_CLUSTERS)
    
    # 클러스터 통계 정보 (DataFrame)
    print("\n" + "=" * 80)
    print("클러스터 통계 정보")
    print("=" * 80)
    stats_df = tracer.get_cluster_statistics()
    print(stats_df.head(15))
    
    # 특정 클러스터의 데이터 확인 예시
    print("\n" + "=" * 80)
    print("특정 클러스터 데이터 확인 예시")
    print("=" * 80)
    
    # 가장 큰 클러스터 확인
    largest_cluster_id = stats_df.iloc[0]['cluster_id']
    print(f"\n가장 큰 클러스터 ID: {largest_cluster_id}")
    cluster_data = tracer.get_cluster_data(largest_cluster_id)
    print(f"데이터 개수: {len(cluster_data)}")
    print("\n첫 3개 샘플:")
    for i, data in enumerate(cluster_data[:3]):
        print(f"\n샘플 {i+1}:")
        if isinstance(data, dict):
            key_fields = ['processName', 'eventName', 'syscall', 'eventId', 'timestamp']
            for field in key_fields:
                if field in data:
                    print(f"  {field}: {data[field]}")
    
    # 클러스터 매핑 정보 저장
    tracer.export_cluster_mapping('cluster_mapping.json')
    
    print("\n" + "=" * 80)
    print("사용 가능한 메서드:")
    print("=" * 80)
    print("1. tracer.get_cluster_data(cluster_id) - 특정 클러스터의 모든 데이터 반환")
    print("2. tracer.get_cluster_summary() - 클러스터별 요약 정보 딕셔너리")
    print("3. tracer.print_cluster_summary(top_n=10) - 클러스터 요약 정보 출력")
    print("4. tracer.get_cluster_statistics() - 클러스터 통계 DataFrame")
    print("5. tracer.export_cluster_mapping(path) - 클러스터 매핑 정보를 JSON으로 저장")
    
else:
    print("embeddings_tensor 또는 train_normal_loader가 아직 생성되지 않았습니다.")
    print("먼저 셀 4를 실행해주세요.")

클러스터링 수행 중...

클러스터링 완료!
클러스터 수: 10
노이즈 포인트: 13
실루엣 점수: 1.0000

클러스터 추적 객체 생성 중...
클러스터 요약 정보
총 클러스터 수: 11
총 데이터 수: 721

[클러스터 8] 데이터 개수: 258
  샘플 데이터: {'processName': 'init', 'eventName': 'setresgid', 'syscall': 'setresgid', 'eventId': '119'}

[클러스터 9] 데이터 개수: 258
  샘플 데이터: {'processName': 'init', 'eventName': 'setresuid', 'syscall': 'setresuid', 'eventId': '117'}

[클러스터 0] 데이터 개수: 62
  샘플 데이터: {'processName': 'initd', 'eventName': 'security_socket_connect', 'syscall': 'connect', 'eventId': '736'}

[클러스터 2] 데이터 개수: 53
  샘플 데이터: {'processName': 'bash', 'eventName': 'setpgid', 'syscall': 'setpgid', 'eventId': '109'}

[클러스터 5] 데이터 개수: 36
  샘플 데이터: {'processName': 'ls', 'eventName': 'security_socket_connect', 'syscall': 'connect', 'eventId': '736'}

[클러스터 1] 데이터 개수: 16
  샘플 데이터: {'processName': 'docker-desktop-', 'eventName': 'security_socket_connect', 'syscall': 'connect', 'eventId': '736'}

[클러스터 3] 데이터 개수: 15
  샘플 데이터: {'processName': 'ls', 'eventName': 'sched_process_exec', 'syscall':

In [81]:
import json
from collections import Counter

def create_ngram_sequence(matched_sequence_file='matched_sequence.json', n=None):
    """
    matched_sequence.json에서 cluster_id를 추출하여 n-gram 시퀀스를 생성하는 함수
    
    Args:
        matched_sequence_file: matched_sequence.json 파일 경로
        n: n-gram의 n 값 (예: 2=bigram, 3=trigram)
            None이면 전역 N_GRAM_SIZE 사용 (.env에서 로드)
    
    Returns:
        list: n-gram 시퀀스 리스트 (각 항목은 n개의 cluster_id 튜플)
    """
    # n 값이 없으면 전역 N_GRAM_SIZE 사용
    if n is None:
        n = globals().get('N_GRAM_SIZE', 2)
    
    # matched_sequence.json 파일 읽기
    with open(matched_sequence_file, 'r', encoding='utf-8') as f:
        matched_sequence = json.load(f)
    
    # cluster_id 시퀀스 추출 (인덱스 순서대로 정렬)
    cluster_sequence = []
    for item in sorted(matched_sequence, key=lambda x: x['index']):
        cluster_sequence.append(item['cluster_id'])
    
    # n-gram 생성
    ngrams = []
    for i in range(len(cluster_sequence) - n + 1):
        ngram = tuple(cluster_sequence[i:i+n])
        ngrams.append(ngram)
    
    return ngrams, cluster_sequence

def analyze_ngrams(ngrams, top_k=10):
    """
    n-gram 통계 분석
    
    Args:
        ngrams: n-gram 리스트
        top_k: 출력할 상위 k개 n-gram
    
    Returns:
        dict: 통계 정보
    """
    # n-gram 빈도 계산
    ngram_counts = Counter(ngrams)
    
    # 통계 정보
    stats = {
        'total_ngrams': len(ngrams),
        'unique_ngrams': len(ngram_counts),
        'top_ngrams': ngram_counts.most_common(top_k),
        'ngram_counts': dict(ngram_counts)
    }
    
    return stats

# n-gram 시퀀스 생성 (예시: bigram, trigram)
print("=" * 80)
print("N-gram 시퀀스 생성")
print("=" * 80)

# N-gram 생성 (.env에서 설정된 N_GRAM_SIZE 사용)
print(f"\n1. {N_GRAM_SIZE}-gram 생성 중...")
ngrams, cluster_seq = create_ngram_sequence('matched_sequence.json', n=None)  # None이면 N_GRAM_SIZE 사용
print(f"   총 {len(ngrams)}개의 {N_GRAM_SIZE}-gram 생성됨")
print(f"   클러스터 시퀀스 길이: {len(cluster_seq)}")

# N-gram 통계
print(f"\n2. {N_GRAM_SIZE}-gram 통계:")
print("-" * 80)
ngram_stats = analyze_ngrams(ngrams, top_k=10)
print(f"   고유 {N_GRAM_SIZE}-gram 수: {ngram_stats['unique_ngrams']}")
print(f"\n   상위 10개 빈도 {N_GRAM_SIZE}-gram:")
for i, (ngram, count) in enumerate(ngram_stats['top_ngrams'], 1):
    print(f"   {i:2d}. {ngram}: {count:4d}회 ({count/len(ngrams)*100:.2f}%)")

# 추가: Bigram과 Trigram도 생성 (비교용)
print("\n3. 추가 통계 (Bigram, Trigram):")
print("-" * 80)
bigrams, _ = create_ngram_sequence('matched_sequence.json', n=2)
trigrams, _ = create_ngram_sequence('matched_sequence.json', n=3)

bigram_stats = analyze_ngrams(bigrams, top_k=5)
trigram_stats = analyze_ngrams(trigrams, top_k=5)

print(f"   Bigram: 총 {len(bigrams)}개, 고유 {bigram_stats['unique_ngrams']}개")
print(f"   Trigram: 총 {len(trigrams)}개, 고유 {trigram_stats['unique_ngrams']}개")

# 클러스터 시퀀스 샘플 출력
print(f"\n4. 클러스터 시퀀스 샘플 (처음 20개):")
print("-" * 80)
print(f"   {cluster_seq[:20]}")

# N-gram 샘플 출력
print(f"\n5. {N_GRAM_SIZE}-gram 샘플 (처음 10개):")
print("-" * 80)
for i, ngram in enumerate(ngrams[:10], 1):
    print(f"   {i:2d}. {ngram}")

print("\n" + "=" * 80)
print("N-gram 생성 완료!")
print("=" * 80)
print(f"\n사용 가능한 변수:")
print(f"  - ngrams: {len(ngrams)}개의 {N_GRAM_SIZE}-gram (기본값, .env에서 설정)")
print(f"  - bigrams: {len(bigrams)}개의 bigram")
print(f"  - trigrams: {len(trigrams)}개의 trigram")
print(f"  - cluster_seq: 원본 클러스터 시퀀스 ({len(cluster_seq)}개)")
print(f"  - ngram_stats: {N_GRAM_SIZE}-gram 통계 정보")
print(f"  - bigram_stats: bigram 통계 정보")
print(f"  - trigram_stats: trigram 통계 정보")

N-gram 시퀀스 생성

1. 5-gram 생성 중...
   총 717개의 5-gram 생성됨
   클러스터 시퀀스 길이: 721

2. 5-gram 통계:
--------------------------------------------------------------------------------
   고유 5-gram 수: 124

   상위 10개 빈도 5-gram:
    1. (5, 6, 6, 5, 5):  125회 (17.43%)
    2. (6, 6, 5, 5, 6):  125회 (17.43%)
    3. (6, 5, 5, 6, 6):  125회 (17.43%)
    4. (5, 5, 6, 6, 5):  125회 (17.43%)
    5. (0, 0, 0, 0, 0):   18회 (2.51%)
    6. (2, 2, 2, 2, 2):   14회 (1.95%)
    7. (2, 2, 3, 4, 4):    9회 (1.26%)
    8. (2, 3, 4, 4, 4):    9회 (1.26%)
    9. (3, 4, 4, 4, 4):    9회 (1.26%)
   10. (0, 0, 0, 2, 2):    4회 (0.56%)

3. 추가 통계 (Bigram, Trigram):
--------------------------------------------------------------------------------
   Bigram: 총 720개, 고유 56개
   Trigram: 총 719개, 고유 81개

4. 클러스터 시퀀스 샘플 (처음 20개):
--------------------------------------------------------------------------------
   [0, 7, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 8, 0, 0]

5. 5-gram 샘플 (처음 10개):
---------------------------------------------

In [83]:
from pybloom import BloomFilter
import json

def create_ngram_bloom_filter(ngrams, error_rate=None):
    """
    N-gram 리스트로부터 Bloom filter를 생성하는 함수
    
    Args:
        ngrams: n-gram 리스트 (튜플들의 리스트)
        error_rate: False positive 확률 (None이면 전역 BLOOM_FILTER_ERROR_RATE 사용)
                    기본값: 0.001 = 0.1%
                    범위: 0 < error_rate < 1
                    주의: 매우 작은 값(예: 1e-100)은 메모리 사용량을 극도로 증가시킬 수 있음
    
    Returns:
        BloomFilter: 생성된 Bloom filter 객체
    """
    # error_rate가 없으면 전역 BLOOM_FILTER_ERROR_RATE 사용
    if error_rate is None:
        error_rate = globals().get('BLOOM_FILTER_ERROR_RATE', 0.001)
    
    # error_rate 유효성 검증
    if error_rate <= 0 or error_rate >= 1:
        raise ValueError(
            f"error_rate는 0과 1 사이의 값이어야 합니다. 현재 값: {error_rate}"
        )
    
    # 매우 작은 값에 대한 경고 (1e-10보다 작으면)
    if error_rate < 1e-10:
        import warnings
        warnings.warn(
            f"매우 작은 error_rate ({error_rate})를 사용하면 메모리 사용량이 극도로 증가할 수 있습니다. "
            f"일반적으로 0.001 (0.1%) 이상의 값을 권장합니다.",
            UserWarning
        )
    
    # 고유 n-gram 개수 계산
    unique_ngrams = set(ngrams)
    capacity = len(unique_ngrams)
    
    # Bloom filter 생성
    bloom = BloomFilter(capacity=capacity, error_rate=error_rate)
    
    # 모든 고유 n-gram을 Bloom filter에 추가
    for ngram in unique_ngrams:
        # 튜플을 문자열로 변환하여 추가
        bloom.add(str(ngram))
    
    return bloom

def check_ngram_in_bloom(bloom_filter, ngram):
    """
    N-gram이 Bloom filter에 존재하는지 확인하는 함수
    
    Args:
        bloom_filter: BloomFilter 객체
        ngram: 확인할 n-gram (튜플)
    
    Returns:
        bool: True면 존재 가능 (False positive 가능), False면 확실히 없음
    """
    return str(ngram) in bloom_filter

def check_ngrams_batch(bloom_filter, ngrams):
    """
    여러 N-gram을 한 번에 체크하는 함수
    
    Args:
        bloom_filter: BloomFilter 객체
        ngrams: 확인할 n-gram 리스트
    
    Returns:
        list: 각 n-gram의 존재 여부 (True/False)
    """
    results = []
    for ngram in ngrams:
        results.append(check_ngram_in_bloom(bloom_filter, ngram))
    return results

def analyze_bloom_filter_performance(bloom_filter, known_ngrams, test_ngrams):
    """
    Bloom filter의 성능을 분석하는 함수
    
    Args:
        bloom_filter: BloomFilter 객체
        known_ngrams: Bloom filter에 추가된 n-gram 리스트 (실제로 존재해야 함)
        test_ngrams: 테스트할 n-gram 리스트
    
    Returns:
        dict: 성능 통계
    """
    known_set = set(known_ngrams)
    test_set = set(test_ngrams)
    
    # True Positive: 실제 존재하고 Bloom filter도 True
    tp = sum(1 for ngram in test_ngrams if ngram in known_set and check_ngram_in_bloom(bloom_filter, ngram))
    
    # False Positive: 실제로 없지만 Bloom filter가 True
    fp = sum(1 for ngram in test_ngrams if ngram not in known_set and check_ngram_in_bloom(bloom_filter, ngram))
    
    # True Negative: 실제로 없고 Bloom filter도 False
    tn = sum(1 for ngram in test_ngrams if ngram not in known_set and not check_ngram_in_bloom(bloom_filter, ngram))
    
    # False Negative: 실제로 존재하지만 Bloom filter가 False (이론적으로 발생하지 않아야 함)
    fn = sum(1 for ngram in test_ngrams if ngram in known_set and not check_ngram_in_bloom(bloom_filter, ngram))
    
    total = len(test_ngrams)
    
    stats = {
        'total_tested': total,
        'true_positive': tp,
        'false_positive': fp,
        'true_negative': tn,
        'false_negative': fn,
        'false_positive_rate': fp / total if total > 0 else 0.0,
        'precision': tp / (tp + fp) if (tp + fp) > 0 else 0.0,
        'recall': tp / (tp + fn) if (tp + fn) > 0 else 0.0
    }
    
    return stats

# N-gram Bloom Filter 생성 및 테스트
print("=" * 80)
print("N-gram Bloom Filter 생성 및 테스트")
print("=" * 80)

# ngrams가 존재하는지 확인
if 'ngrams' in globals() and ngrams:
    print(f"\n1. Bloom Filter 생성 중...")
    print(f"   입력 N-gram 개수: {len(ngrams)}")
    print(f"   고유 N-gram 개수: {len(set(ngrams))}")
    
    # Bloom filter 생성 (.env에서 설정된 error_rate 사용)
    bloom_filter = create_ngram_bloom_filter(ngrams, error_rate=None)  # None이면 BLOOM_FILTER_ERROR_RATE 사용
    print(f"   Bloom Filter 생성 완료!")
    print(f"   Bloom Filter 크기: {bloom_filter.capacity}개 용량")
    print(f"   False Positive Rate: {bloom_filter.error_rate * 100:.3f}%")
    
    # 샘플 체크
    print(f"\n2. 샘플 N-gram 체크:")
    print("-" * 80)
    sample_ngrams = ngrams[:10]
    for i, ngram in enumerate(sample_ngrams, 1):
        exists = check_ngram_in_bloom(bloom_filter, ngram)
        print(f"   {i:2d}. {ngram}: {'존재함' if exists else '존재하지 않음'}")
    
    # 성능 분석
    print(f"\n3. Bloom Filter 성능 분석:")
    print("-" * 80)
    # 전체 ngrams를 train/test로 분할 (80/20)
    split_idx = int(len(ngrams) * 0.8)
    train_ngrams = ngrams[:split_idx]
    test_ngrams = ngrams[split_idx:]
    
    # Train set으로 Bloom filter 재생성 (.env에서 설정된 error_rate 사용)
    train_bloom = create_ngram_bloom_filter(train_ngrams, error_rate=None)  # None이면 BLOOM_FILTER_ERROR_RATE 사용
    
    # Test set으로 성능 측정
    performance = analyze_bloom_filter_performance(train_bloom, train_ngrams, test_ngrams)
    
    print(f"   테스트 N-gram 개수: {performance['total_tested']}")
    print(f"   True Positive: {performance['true_positive']}")
    print(f"   False Positive: {performance['false_positive']} ({performance['false_positive_rate']*100:.3f}%)")
    print(f"   True Negative: {performance['true_negative']}")
    print(f"   False Negative: {performance['false_negative']}")
    print(f"   Precision: {performance['precision']*100:.2f}%")
    print(f"   Recall: {performance['recall']*100:.2f}%")
    
    # 새로운 N-gram 체크 예시
    print(f"\n4. 새로운 N-gram 체크 예시:")
    print("-" * 80)
    # 존재하지 않을 가능성이 높은 새로운 n-gram 생성
    max_cluster_id = max([max(ng) for ng in ngrams if ng])
    new_ngrams = [
        tuple([max_cluster_id + 100 + i for i in range(N_GRAM_SIZE)]),  # 존재하지 않을 n-gram
        ngrams[0] if ngrams else tuple([0] * N_GRAM_SIZE),  # 존재하는 n-gram
    ]
    
    for ngram in new_ngrams:
        exists = check_ngram_in_bloom(bloom_filter, ngram)
        print(f"   {ngram}: {'존재 가능 (False positive 가능)' if exists else '존재하지 않음'}")
    
    print("\n" + "=" * 80)
    print("Bloom Filter 생성 완료!")
    print("=" * 80)
    print(f"\n사용 가능한 변수:")
    print(f"  - bloom_filter: 생성된 Bloom filter 객체")
    print(f"  - train_bloom: Train set으로 생성된 Bloom filter")
    print(f"\n사용 예시:")
    print(f"  - check_ngram_in_bloom(bloom_filter, (0, 1))")
    print(f"  - check_ngrams_batch(bloom_filter, [(0,1), (1,2), (2,3)])")
    
else:
    print("ngrams 변수가 없습니다. 먼저 셀 8을 실행하여 n-gram을 생성해주세요.")

ModuleNotFoundError: No module named 'pybloom'

In [82]:
# refine_cluster_mapping 함수는 dataAnalyze.ipynb로 이동되었습니다.
# dataAnalyze.ipynb 파일에서 함수를 import하거나 직접 사용하세요.